In [1]:
%run 0-Base.py

In [2]:
SOURCE_PATH = f'{DATA_PATH}/3-joined'
TARGET_PATH = f'{DATA_PATH}/4-feature-engineered'

In [3]:
%time historical_transactions_df   = pd.read_feather(f'{SOURCE_PATH}/historical_transactions.feather')
%time new_merchant_transactions_df = pd.read_feather(f'{SOURCE_PATH}/new_merchant_transactions.feather')

CPU times: user 1.07 s, sys: 178 ms, total: 1.25 s
Wall time: 521 ms
CPU times: user 255 ms, sys: 29.6 ms, total: 285 ms
Wall time: 240 ms


In [4]:
%time transactions_df = pd.concat([historical_transactions_df, new_merchant_transactions_df])

del historical_transactions_df
del new_merchant_transactions_df

CPU times: user 2.41 s, sys: 415 ms, total: 2.82 s
Wall time: 2.82 s


In [5]:
%time transactions_df[    'authorized_purchase_amount'] = transactions_df.purchase_amount.where( transactions_df.authorized_flag)
%time transactions_df['not_authorized_purchase_amount'] = transactions_df.purchase_amount.where(~transactions_df.authorized_flag)

CPU times: user 90.1 ms, sys: 24.5 ms, total: 115 ms
Wall time: 113 ms
CPU times: user 106 ms, sys: 19.5 ms, total: 125 ms
Wall time: 124 ms


In [6]:
%%time

transactions_df['purchase_year' ] = transactions_df.purchase_date.dt.year
transactions_df['purchase_month'] = transactions_df.purchase_date.dt.month

CPU times: user 1.86 s, sys: 71.4 ms, total: 1.93 s
Wall time: 1.93 s


In [7]:
%time transactions_df = pd.get_dummies(transactions_df, columns=['installments'], dtype='int8')

CPU times: user 4.06 s, sys: 833 ms, total: 4.89 s
Wall time: 4.89 s


In [8]:
agg = {
    'authorized_flag': ['count', 'mean'],
    
    'purchase_year':  ['first'],
    'purchase_month': ['first'],
    
    'purchase_amount':                ['min', 'mean', 'max'],
    'authorized_purchase_amount':     ['min', 'mean', 'max'],
    'not_authorized_purchase_amount': ['min', 'mean', 'max'],
    
    'installments_-1':  ['mean'],
    'installments_0':   ['mean'],
    'installments_1':   ['mean'],
    'installments_2':   ['mean'],
    'installments_3':   ['mean'],
    'installments_4':   ['mean'],
    'installments_5':   ['mean'],
    'installments_6':   ['mean'],
    'installments_7':   ['mean'],
    'installments_8':   ['mean'],
    'installments_9':   ['mean'],
    'installments_10':  ['mean'],
    'installments_11':  ['mean'],
    'installments_12':  ['mean'],
    'installments_999': ['mean'],
}

In [9]:
%time aggregated_transactions_df = transactions_df.groupby(['card_id', 'month_lag']).agg(agg)
aggregated_transactions_df.columns = [f'{fn}({col})' for col, fn in aggregated_transactions_df.columns]
aggregated_transactions_df = aggregated_transactions_df.rename(columns={'count(authorized_flag)': 'count'}).reset_index()

with pd.option_context('display.max_columns', 1000): display(aggregated_transactions_df)

CPU times: user 18.3 s, sys: 2.02 s, total: 20.3 s
Wall time: 20.3 s


,card_id,month_lag,count,mean(authorized_flag),first(purchase_year),first(purchase_month),min(purchase_amount),mean(purchase_amount),max(purchase_amount),min(authorized_purchase_amount),mean(authorized_purchase_amount),max(authorized_purchase_amount),min(not_authorized_purchase_amount),mean(not_authorized_purchase_amount),max(not_authorized_purchase_amount),mean(installments_-1),mean(installments_0),mean(installments_1),mean(installments_2),mean(installments_3),mean(installments_4),mean(installments_5),mean(installments_6),mean(installments_7),mean(installments_8),mean(installments_9),mean(installments_10),mean(installments_11),mean(installments_12),mean(installments_999)
0,C_ID_00007093c1,-12,4,1.000000,2017,2,-0.572600,-0.333604,0.037176,-0.572600,-0.333604,0.037176,NaN,NaN,NaN,0.000000,0.0,0.500000,0.250000,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C_ID_00007093c1,-11,10,0.700000,2017,3,-0.677936,-0.571263,-0.401298,-0.677936,-0.560815,-0.401298,-0.671775,-0.595641,-0.557574,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C_ID_00007093c1,-10,12,0.750000,2017,4,-0.728876,-0.542391,0.004418,-0.728876,-0.546012,0.004418,-0.671775,-0.531528,-0.311139,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C_ID_00007093c1,-9,10,0.600000,2017,5,-0.698823,-0.342525,0.307953,-0.698823,-0.574854,-0.296112,-0.596447,0.005969,0.307953,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C_ID_00007093c1,-8,19,0.684211,2017,6,-0.728876,-0.519404,0.004418,-0.721363,-0.577617,-0.234504,-0.728876,-0.393276,0.004418,0.000000,0.0,0.684211,0.105263,0.210526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044123,C_ID_fffffd5772,-3,12,1.000000,2017,11,-0.728876,-0.694185,-0.518505,-0.728876,-0.694185,-0.518505,NaN,NaN,NaN,0.083333,0.0,0.916667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3044124,C_ID_fffffd5772,-2,22,1.000000,2017,12,-0.731881,-0.694078,-0.535154,-0.731881,-0.694078,-0.535154,NaN,NaN,NaN,0.000000,0.0,0.909091,0.090909,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3044125,C_ID_fffffd5772,-1,10,0.900000,2018,1,-0.728876,-0.692835,-0.581616,-0.728876,-0.688830,-0.581616,-0.728876,-0.728876,-0.728876,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3044126,C_ID_fffffd5772,0,24,0.666667,2018,2,-0.745405,-0.697689,-0.311139,-0.745405,-0.677963,-0.311139,-0.745405,-0.737141,-0.728876,0.000000,0.0,0.958333,0.000000,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
%time aggregated_transactions_df.to_feather(f'{TARGET_PATH}/aggregated-transactions-by-card-id.feather')

CPU times: user 497 ms, sys: 388 ms, total: 885 ms
Wall time: 2.78 s
